In [1]:
!git clone https://github.com/comfyanonymous/ComfyUI.git /workspace/ComfyUI


Cloning into '/workspace/ComfyUI'...
remote: Enumerating objects: 28427, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 28427 (delta 48), reused 27 (delta 22), pack-reused 28340 (from 3)
Receiving objects: 100% (28427/28427), 69.03 MiB | 16.18 MiB/s, done.
Resolving deltas: 100% (19247/19247), done.
Updating files: 100% (587/587), done.


In [2]:
# === INSTALLATION COMFYUI-MANAGER ===

import subprocess

print(">>> Installation de ComfyUI-Manager...")

subprocess.run("git clone https://github.com/Comfy-Org/ComfyUI-Manager /workspace/ComfyUI/custom_nodes/ComfyUI-Manager", shell=True)

# Installation des dépendances
subprocess.run("source /workspace/ComfyUI/venv/bin/activate && pip install -r /workspace/ComfyUI/custom_nodes/ComfyUI-Manager/requirements.txt", shell=True)

print("\n>>> ComfyUI-Manager installé.")
print(">>> Relance ComfyUI pour que le bouton 'Manager' apparaisse dans l'interface.")



>>> Installation de ComfyUI-Manager...


Cloning into '/workspace/ComfyUI/custom_nodes/ComfyUI-Manager'...
Updating files: 100% (96/96), done.



>>> ComfyUI-Manager installé.
>>> Relance ComfyUI pour que le bouton 'Manager' apparaisse dans l'interface.


/bin/sh: 1: source: not found


In [6]:
# QWEN FILES DOWNLOAD

import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. CONFIGURATION SÉCURISÉE ---
CIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN", "")
HF_TOKEN = os.environ.get("HF_TOKEN", "")

# Installation silencieuse de aria2
os.system("apt-get update -y > /dev/null && apt-get install -y aria2 > /dev/null")

# --- 2. BASE DE DONNÉES (Avec Tailles) ---
db_models = {
    "🐲 Qwen Core (Image & Edit)": [
        ("[38.1 GB] Qwen Image BF16", "https://huggingface.co/Comfy-Org/Qwen2.5-VL-ComfyUI-Repackaged/resolve/main/qwen_image_bf16.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "qwen_image_bf16.safetensors", "hf"),
        ("[19.0 GB] Qwen Edit Model (FP8)", "https://huggingface.co/aidiffuser/Qwen-Image-Edit-2509/resolve/main/Qwen-Image-Edit-2509_fp8_e4m3fn.safetensors?download=true", "/workspace/ComfyUI/models/diffusion_models", "Qwen-Image-Edit-2509_fp8_e4m3fn.safetensors", "hf"),
        ("[242 MB] Qwen VAE", "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/vae/qwen_image_vae.safetensors", "/workspace/ComfyUI/models/vae", "qwen_image_vae.safetensors", "hf"),
        ("[9.4 GB] Qwen CLIP (VL 7B) (FP8) Scaled", "https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/text_encoders/qwen_2.5_vl_7b_fp8_scaled.safetensors", "/workspace/ComfyUI/models/text_encoders", "qwen_2.5_vl_7b_fp8_scaled.safetensors", "hf"),
    ],
    "🎨 Qwen LoRAs (Noms Complets)": [
        ("[1.6 GB] Qwen-Image-Lightning-8steps-V2.0-bf16", "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Lightning-8steps-V2.0-bf16.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "Qwen-Image-Lightning-8steps-V2.0-bf16.safetensors", "hf"),
        ("[810 MB] Qwen-Image-Edit-Lightning-8steps-V1.0-bf16", "https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Edit-Lightning-8steps-V1.0-bf16.safetensors?download=true", "/workspace/ComfyUI/models/loras/QWEN", "Qwen-Image-Edit-Lightning-8steps-V1.0-bf16.safetensors", "hf"),
        ("[1.1 GB] Famegrid SkinFix (Civitai)", "https://civitai.com/api/download/models/2453097?type=Model&format=SafeTensor", "/workspace/ComfyUI/models/loras/QWEN", "Famegrid_Qwen_Lora_Standard_V1.5_RealSkinFix.safetensors", "civitai"),
        ("[2.2 GB] 1Girl Qwen Concept", "https://huggingface.co/dci05049/qwen/resolve/main/1girl-qwen_v3.safetensors", "/workspace/ComfyUI/models/loras/QWEN", "1girl-qwen_v3.safetensors", "hf"),
    ]
}

# --- 3. INTERFACE GRAPHIQUE ---
style = {'description_width': 'initial'}
layout = widgets.Layout(width='98%')
checkboxes = {}
output_log = widgets.Output(layout={'border': '1px solid #444', 'height': '200px', 'overflow_y': 'scroll'})

accordion_children = []
titles = []

for category, items in db_models.items():
    vbox_items = []
    select_all = widgets.Button(description="Tout cocher", icon="check-square-o", button_style='info', layout=widgets.Layout(width='150px'))
    def on_select_all(b, cat=category):
        for k, val in checkboxes.items():
            if k.startswith(cat):
                val['cb'].value = True
    select_all.on_click(on_select_all)
    vbox_items.append(select_all)
    
    for item in items:
        name, url, path, filename, src = item
        key = f"{category}|{filename}"
        label = f"{name}"
        cb = widgets.Checkbox(value=False, description=label, style=style, layout=layout)
        checkboxes[key] = {'cb': cb, 'data': item}
        vbox_items.append(cb)
        
    accordion_children.append(widgets.VBox(vbox_items))
    titles.append(category)

accordion = widgets.Accordion(children=accordion_children)
for i, title in enumerate(titles):
    accordion.set_title(i, title)
    accordion.selected_index = 0

btn_download = widgets.Button(
    description='LANCER LE TÉLÉCHARGEMENT (Qwen)',
    button_style='success', 
    layout=widgets.Layout(width='100%', height='60px', margin='20px 0px 0px 0px'),
    icon='download'
)

# --- 4. LOGIQUE DE TÉLÉCHARGEMENT ---
def run_download(b):
    output_log.clear_output()
    with output_log:
        print("🚀 Traitement des fichiers Qwen...\n")
        count = 0
        for key, val in checkboxes.items():
            if val['cb'].value:
                count += 1
                item = val['data']
                name, url, path, filename, src = item
                
                os.makedirs(path, exist_ok=True)
                full_path = os.path.join(path, filename)
                
                if os.path.exists(full_path) and os.path.getsize(full_path) > 1024:
                    print(f"⏩ [SKIP] {name} est déjà là. (0 data utilisée)")
                    continue

                auth = ""
                final_url = url
                if src == "hf":
                    auth = f'--header="Authorization: Bearer {HF_TOKEN}"'
                elif src == "civitai":
                    sep = "&" if "?" in url else "?"
                    final_url = f"{url}{sep}token={CIVITAI_TOKEN}"
                
                print(f"⬇️ Téléchargement de : {name}")
                cmd = f'aria2c -x 16 -s 16 -k 1M -c {auth} -d "{path}" -o "{filename}" "{final_url}"'
                ret = os.system(cmd)
                if ret == 0: print(f"   ✅ OK.")
                else: print(f"   ⚠️ Erreur (Code {ret}).")
                    
        if count == 0: print("⚠️ Aucune case cochée.")
        else: print("\n🎉 Terminé.")

btn_download.on_click(run_download)

display(widgets.VBox([
    widgets.Label("🐲 INSTALLATION QWEN 2.5 (Avec Tailles)"),
    accordion,
    btn_download,
    output_log
]))

In [ ]:
# DISABLE REACTOR NSFW FILTER

import os

# Le chemin exact indiqué dans le guide (reactor_sfw.py)
target_path = "/workspace/ComfyUI/custom_nodes/comfyui-reactor/scripts/reactor_sfw.py"

# Le code patché (Nettoyé des commentaires qui cassaient la syntaxe)
bypass_code = """
from PIL import Image
import logging
import os
# On garde les imports essentiels pour éviter que ReActor ne plante au démarrage
from scripts.reactor_logger import logger

# On déclare que le modèle existe pour empêcher le téléchargement
MODEL_EXISTS = True

def ensure_nsfw_model(nsfwdet_model_path):
    # On ment au système : "Oui, le modèle est là, tout va bien"
    return True

def nsfw_image(img_data, model_path: str):
    # C'est ici que le bypass agit.
    # False = "Ce n'est pas NSFW" (Safe)
    # On renvoie False systématiquement sans même regarder l'image.
    return False
"""

print(f"🔧 Application du patch sur : {target_path}")

# Vérification d'existence du dossier
folder = os.path.dirname(target_path)
if os.path.exists(folder):
    try:
        with open(target_path, "w") as f:
            f.write(bypass_code)
        print("✅ Fichier 'reactor_sfw.py' écrasé avec succès.")
        print("🔓 Le filtre NSFW est maintenant désactivé.")
    except Exception as e:
        print(f"❌ Erreur lors de l'écriture : {e}")
else:
    print(f"❌ Le dossier {folder} n'existe pas. Vérifie l'installation de ReActor.")

# Par sécurité, on patche aussi l'ancien nom de fichier s'il existe (certaines versions varient)
alt_path = "/workspace/ComfyUI/custom_nodes/comfyui-reactor/scripts/reactor_nsfw.py"
if os.path.exists(alt_path):
    with open(alt_path, "w") as f:
        f.write(bypass_code)
    print("✅ Ancien fichier 'reactor_nsfw.py' également patché par sécurité.")

print("\n🚨 IMPORTANT : Tu dois REDÉMARRER ComfyUI (Stop/Start) pour que ce changement prenne effet !")

In [5]:
# DÉMARRAGE COMFYUI (TOUT-EN-UN : FIX PYTORCH + DÉPENDANCES)

import os
import subprocess
import sys

# --- CONFIGURATION ---
COMFY_DIR = "/workspace/ComfyUI"
MANAGER_DIR = f"{COMFY_DIR}/custom_nodes/ComfyUI-Manager"

print("🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...")

# 1. INSTALLATION OUTILS SYSTÈME
print("\n🔧 [1/6] Vérification outils système (ffmpeg, aria2)...")
subprocess.run(["apt-get", "update"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run(["apt-get", "install", "-y", "ffmpeg", "aria2"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 2. RÉPARATION PYTORCH (Le Fix Critique)
# On force la version 2.5.1 pour éviter l'erreur 'is_compiling'
print("\n🔥 [2/6] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...")
# On vire xformers qui cause souvent des conflits avec les nouvelles versions
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "xformers"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
# Installation de la version stable CUDA 12.4
subprocess.check_call([
    sys.executable, "-m", "pip", "install",
    "torch==2.5.1+cu124", "torchvision==0.20.1+cu124", "torchaudio==2.5.1+cu124",
    "--index-url", "https://download.pytorch.org/whl/cu124"
])

# 3. INSTALLATION DÉPENDANCES PYTHON
print("\n📦 [3/6] Installation des librairies manquantes...")
required_packages = [
    "numpy<2",           # Force NumPy 1.x (Crucial)
    "pydantic-settings", # Fix config
    "sentencepiece",     # Fix T5/Wan/Flux
    "aiohttp",           # Fix Manager
    "einops",            # Fix Core
    "av",                # Fix Vidéo
    "torchsde",          # Fix Samplers
    "scipy",             # Fix Math
    "insightface",       # Fix FaceSwap
    "onnxruntime-gpu",   # Fix FaceSwap
    "kornia",            # Fix Image Proc
    "spandrel",          # Fix Upscale
    "color-matcher"      # Fix Wan Color
]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + required_packages)

# Requirements du Manager
if os.path.exists(f"{MANAGER_DIR}/requirements.txt"):
    print("   -> Dépendances ComfyUI-Manager...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", f"{MANAGER_DIR}/requirements.txt"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 4. TÉLÉCHARGEMENT MODÈLES INSIGHTFACE (AntelopeV2)
print("\n📥 [4/6] Vérification des modèles InsightFace...")
antelope_dir = f"{COMFY_DIR}/models/insightface/models/antelopev2"
os.makedirs(antelope_dir, exist_ok=True)
files_antelope = ["1k3d68.onnx", "2d106det.onnx", "genderage.onnx", "glintr100.onnx", "scrfd_10g_bnkps.onnx"]

for f in files_antelope:
    if not os.path.exists(f"{antelope_dir}/{f}"):
        print(f"   ⬇️ Téléchargement : {f}")
        subprocess.run(["aria2c", "-x", "16", "-s", "16", "-k", "1M", "-c", f"https://huggingface.co/okaris/antelopev2/resolve/main/{f}?download=true", "-d", antelope_dir, "-o", f], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 5. CONFIGURATION FINALE
print("\n⚙️ [5/6] Configuration finale...")
# On s'assure que le dossier custom_nodes est propre si besoin (optionnel)

# 6. LANCEMENT
print("\n🚀 [6/6] Lancement du serveur...")

run_script = """#!/bin/bash
cd /workspace/ComfyUI
# Arguments optimisés pour RunPod
python main.py --listen 0.0.0.0 --port 8188 --disable-auto-launch --cache-ram 8 --bf16-unet
"""

with open("run_comfy.sh", "w") as f:
    f.write(run_script)
os.chmod("run_comfy.sh", 0o755)

print("✅ ComfyUI démarre. (Si le terminal bloque ici, c'est bon signe : le serveur tourne !)")
!./run_comfy.sh

🚀 DÉMARRAGE COMFYUI (Mode Réparation & Mise à jour)...

🔧 [1/6] Vérification outils système (ffmpeg, aria2)...

🔥 [2/6] Mise à jour du Noyau PyTorch (Cela prend ~1 min)...
Looking in indexes: https://download.pytorch.org/whl/cu124



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip



📦 [3/6] Installation des librairies manquantes...
  Using cached pydantic_settings-2.12.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached sentencepiece-0.2.1-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached aiohttp-3.13.2-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (8.1 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached av-16.0.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (4.6 kB)
  Using cached torchsde-0.2.6-py3-none-any.whl.metadata (5.3 kB)
  Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached insightface-0.7.3.tar.gz (439 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Prepari


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


   -> Dépendances ComfyUI-Manager...

📥 [4/6] Vérification des modèles InsightFace...
   ⬇️ Téléchargement : 1k3d68.onnx
   ⬇️ Téléchargement : 2d106det.onnx
   ⬇️ Téléchargement : genderage.onnx
   ⬇️ Téléchargement : glintr100.onnx
   ⬇️ Téléchargement : scrfd_10g_bnkps.onnx

⚙️ [5/6] Configuration finale...

🚀 [6/6] Lancement du serveur...
✅ ComfyUI démarre. (Si le terminal bloque ici, c'est bon signe : le serveur tourne !)
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-12-10 22:57:27.574
** Platform: Linux
** Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
** Python executable: /usr/bin/python
** ComfyUI Path: /workspace/ComfyUI
** ComfyUI Base Folder Path: /workspace/ComfyUI
** User directory: /workspace/ComfyUI/user
** ComfyUI-Manager config path: /workspace/ComfyUI/user/__manager/config.ini
** Log path: /workspace/ComfyUI/user/comfyui.log

[notice] A new release of pip is available: 24